In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("..")

import os
import optuna
import torch
import wandb
import gc

from src.training import train_model
from src.models.model_unet import UNet
from src.dataprocess.torch_dataset_SNSD import DenoisingSpectrogramDataset
from torch.utils.data import DataLoader

# Path to Dataset
DATA_DIR = "/media/gabriel/500 BKP/datasets/SNSD-DataSet/preprocessed"
CHECKPOINTS_DIR = os.path.dirname(os.getcwd()) + "/checkpoints"


/home/gabriel/miniconda3/envs/venv_audio/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: gfiuzaa (gfiuzaa-universidade-federal-do-rio-de-janeiro). Use `wandb login --relogin` to force relogin


True

In [4]:
# Dataset
TARGET_FRAMES = 1024
train_dataset = DenoisingSpectrogramDataset(DATA_DIR, 'train', TARGET_FRAMES)
val_dataset = DenoisingSpectrogramDataset(DATA_DIR, 'validation', TARGET_FRAMES)

def objective(trial):
    try:
        # Search Space
        base_filters = trial.suggest_categorical('base_filters', [16, 32])
        depth = trial.suggest_int('depth', 3, 5)
        dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.3)
        lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
        batch_size = trial.suggest_categorical('batch_size', [8, 16])

        # wandb logging
        wandb.init(project='audio-denoising', config={
            'base_filters': base_filters,
            'depth': depth,
            'dropout_rate': dropout_rate,
            'lr': lr,
            'batch_size': batch_size
        }, reinit=True)

        # Dataloader
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, persistent_workers=False)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, persistent_workers=False)

        # Model
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = UNet(
            in_channels=1,
            out_channels=1,
            base_filters=base_filters,
            depth=depth,
            dropout_rate=dropout_rate
        ).to(device)

        # Optimizer and loss
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=2)
        criterion = torch.nn.MSELoss()

        # Train
        model, train_losses, val_losses, best_val_loss = train_model(
            model=model,
            optimizer=optimizer,
            criterion=criterion,
            scheduler=scheduler,
            device=device,
            train_loader=train_loader,
            train_dataset=train_dataset,
            val_loader=val_loader,
            val_dataset=val_dataset,
            num_epochs=100,
            early_stopping_patience=5,
            audio_preview=False,
            trial=trial,
            save_dir=CHECKPOINTS_DIR
        )

    except RuntimeError as e:
        if "out of memory" in str(e):
            print(f"[Trial {trial.number}] - Out of memory. Skipping...")
            return float('inf')  # Penalize trial but don't crash study
        else:
            raise e

    finally:
        # Always clean up
        if 'model' in locals():
            model.cpu()
            del model
        if 'optimizer' in locals():
            del optimizer
        if 'scheduler' in locals():
            del scheduler
        if 'train_loader' in locals():
            del train_loader
        if 'val_loader' in locals():
            del val_loader

        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
        gc.collect()

        wandb.finish()

    return best_val_loss

In [5]:
from joblib import parallel_backend

study = optuna.create_study(
    study_name="audio-denoising",
    direction="minimize",
    storage="sqlite:///optuna_trials.db",
    load_if_exists=True,
)

with parallel_backend('loky'):
    study.optimize(objective, n_trials=50, n_jobs=1, timeout=8*60*60)  # 8 hours

[I 2025-04-27 17:40:57,767] A new study created in RDB with name: audio-denoising



Epoch 1/100


Best model saved: /home/gabriel/VSProjects/audio_analysis/DenoisingAutoencoder/checkpoints/model_trial_0.pth (val loss: 0.5435)
Train Loss: 1.0049 | Val Loss: 0.5435
Learning Rate: 0.000018

Epoch 2/100


Best model saved: /home/gabriel/VSProjects/audio_analysis/DenoisingAutoencoder/checkpoints/model_trial_0.pth (val loss: 0.3232)
Train Loss: 0.4899 | Val Loss: 0.3232
Learning Rate: 0.000018

Epoch 3/100


Best model saved: /home/gabriel/VSProjects/audio_analysis/DenoisingAutoencoder/checkpoints/model_trial_0.pth (val loss: 0.2654)
Train Loss: 0.3515 | Val Loss: 0.2654
Learning Rate: 0.000018

Epoch 4/100


...

No improvement for 3 epoch(s)
Train Loss: 0.0093 | Val Loss: 0.0127
Learning Rate: 0.000003

Epoch 25/100


No improvement for 4 epoch(s)
Train Loss: 0.0092 | Val Loss: 0.0128
Learning Rate: 0.000003

Epoch 26/100


No improvement for 5 epoch(s)
Early stopping triggered.


train_loss,█▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▃▃▂▂▂▂▂▁▁▂▂▁▁█▁▁▁▁▁▁▁▁▁▁▁
train_loss,0.00917
val_loss,0.0128


[I 2025-04-28 01:55:04,954] Trial 16 finished with value: 0.012617661465580264 and parameters: {'base_filters': 32, 'depth': 3, 'dropout_rate': 0.05638063453006781, 'lr': 0.0002951929866638849, 'batch_size': 8}. Best is trial 13 with value: 0.011547641990085443.


In [10]:
import optuna
import pandas as pd

def export_study_to_csv(study_path, study_name, out_csv="optuna_trials.csv"):
    study = optuna.load_study(study_name=study_name, storage=study_path)

    rows = []
    for trial in study.trials:
        row = {
            "trial": trial.number,
            "val_loss": trial.value,
            "model_path": trial.user_attrs.get("best_model_path", "N/A")
        }
        # Add hyperparameters
        row.update(trial.params)
        rows.append(row)

    df = pd.DataFrame(rows)
    df = df.sort_values("val_loss")
    df.to_csv(out_csv, index=False)

    print(f"✅ Exported {len(df)} trials to {out_csv}")

    return df

In [11]:
df = export_study_to_csv(
    study_path="sqlite:///optuna_trials.db",
    study_name="audio-denoising",
    out_csv=os.getcwd() + "/optuna_trials.csv"
)

✅ Exported 17 trials to /home/gabriel/VSProjects/audio_analysis/DenoisingAutoencoder/notebooks/optuna_trials.csv


In [12]:
df

,trial,val_loss,model_path,base_filters,depth,dropout_rate,lr,batch_size
13,13,0.011548,/home/gabriel/VSProjects/audio_analysis/Denois...,32,4,0.004557,0.000401,8
15,15,0.012022,/home/gabriel/VSProjects/audio_analysis/Denois...,32,4,0.049452,0.000342,8
6,6,0.012372,/home/gabriel/VSProjects/audio_analysis/Denois...,32,4,0.043183,0.000451,8
16,16,0.012618,/home/gabriel/VSProjects/audio_analysis/Denois...,32,3,0.056381,0.000295,8
10,10,0.013474,/home/gabriel/VSProjects/audio_analysis/Denois...,32,4,0.102038,0.000924,8
14,14,0.013957,/home/gabriel/VSProjects/audio_analysis/Denois...,32,4,0.008029,0.000064,8
5,5,0.015866,/home/gabriel/VSProjects/audio_analysis/Denois...,32,3,0.009160,0.000022,8
12,12,0.016956,/home/gabriel/VSProjects/audio_analysis/Denois...,32,4,0.102544,0.000771,8
11,11,0.022328,/home/gabriel/VSProjects/audio_analysis/Denois...,32,4,0.099940,0.000852,8
2,2,0.023014,/home/gabriel/VSProjects/audio_analysis/Denois...,16,4,0.060515,0.000023,16
